In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/random'
folder = 'sp_matrix_tag_pop'
group_info = 'group_info/high'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [52]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [53]:
predictions[:5]

,uid,tid,rating
0,0,2,0.070515
1,0,14,0.022937
2,0,15,0.147609
3,0,20,0.038996
4,0,21,0.023123


In [54]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [55]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5


In [56]:
num_user = len(predictions['uid'].unique())
num_user

218

In [57]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [58]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.16192660550458718


In [59]:
groups = np.loadtxt(os.path.join(dir_, 'groups', 'random' ,'2.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([37., 29.])

In [60]:
n = 20
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)
    
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
    satisfication += precision / n
print(satisfication/(len(groups)))


0.2816513761467892


In [61]:
top_4_results = []
top_20_results = []
top_100_results = []

In [62]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

In [63]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.3532110091743119, 0.4444444444444444, 0.5740740740740741, 0.6162790697674418]
------------------
[0.2816513761467892, 0.35763888888888906, 0.4259259259259259, 0.5011627906976746]
------------------
[0.20293577981651378, 0.26708333333333334, 0.32833333333333337, 0.38953488372093026]
